In [1]:
import pandas as pd
import time, requests, json
#from selenium import webdriver
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [2]:
#*   Prohibits truncatation of data in prints.
pd.set_option('display.max_colwidth', -1)

In [3]:
allWinAndNom = pd.read_csv("/Users/nicolespaar/Desktop/oscars_winners.csv")

def getWinnersDF_Film(allWinAndNOm):

    allWinners_year = []
    allWinners_category = []
    allWinners_entity = []

    for row in allWinAndNom.iterrows():
        if row[1]['winner'] == True:
            allWinners_year.append(row[1]['year'])
            allWinners_category.append(row[1]['category'])
            allWinners_entity.append(row[1]['entity'])
            
    winners_data = {'Year': allWinners_year, 'Category': allWinners_category, 'Entity': allWinners_entity}
    winnersDF = pd.DataFrame(data=winners_data)
    
    winnersDF_films_A = winnersDF[winnersDF['Category'] == 'BEST PICTURE'].copy()
    winnersDF_films_B = winnersDF[winnersDF['Category'] == 'OUTSTANDING PRODUCTION'].copy()
    winnersDF_films_C = winnersDF[winnersDF['Category'] == 'OUTSTANDING MOTION PICTURE'].copy()
    winnersDF_films_D = winnersDF[winnersDF['Category'] == 'BEST MOTION PICTURE'].copy()
    winnersDF_films_E = winnersDF[winnersDF['Category'] == 'OUTSTANDING PICTURE'].copy()
    
    winners_years = []
    winners_entities = []
    for row in winnersDF_films_E.iterrows():
        winners_years.append(row[1]['Year'])
    winners_entities.append("wings")
    winners_entities.append("the broadway melody")

    for row in winnersDF_films_B.iterrows():
        winners_years.append(row[1]['Year'])
        winners_entities.append(row[1]['Entity'].lower())
    for row in winnersDF_films_C.iterrows():
        winners_years.append(row[1]['Year'])
        winners_entities.append(row[1]['Entity'].lower())
    for row in winnersDF_films_D.iterrows():
        winners_years.append(row[1]['Year'])
        winners_entities.append(row[1]['Entity'].lower())
    for row in winnersDF_films_A.iterrows():
        winners_years.append(row[1]['Year'])
        winners_entities.append(row[1]['Entity'].lower())
    winners_film_data = {'Year': winners_years, 'Entity': winners_entities}

    winnersDF_films = pd.DataFrame(data=winners_film_data)
    
    return winnersDF_films, winners_entities, winners_years

In [4]:
winnersDF_Film, entities, years = getWinnersDF_Film(allWinAndNom)
#winnersDF_Film.head(3)

In [5]:
print("titleBasics..")
#titleBasics = pd.read_csv("/Users/nicolespaar/Desktop/oscarDataAnalysis/Spaar/resources/data/title.basics.tsv", sep='\t', header=0, low_memory=False)
#lowercased_titles = []
#count = 0
#for row in titleBasics.iterrows():
#    try:
#        lower = row[1].primaryTitle.lower()
#        lowercased_titles.append(lower)
#    except:
#        count += 1
#        lowercased_titles.append("n/a")
#print(f"{count} movies had no 'primaryTitle'")
#titleBasics['lowercasedTitle'] = lowercased_titles
#titleBasics.to_csv("titleBasics_spaar.csv")

titleBasics..


In [6]:
titleBasics = pd.read_csv("/Users/nicolespaar/Desktop/titleBasics_spaar.csv", index_col='tconst', low_memory=False)
#titleBasics.head(3)

In [7]:
print("titleCrew..")
titleCrew = pd.read_csv("/Users/nicolespaar/Desktop/title.crew.tsv", sep='\t', header=0)
print("nameBasics..")
nameBasics = pd.read_csv("/Users/nicolespaar/Desktop/name.basics.tsv", sep='\t', header=0, index_col='nconst')
("titlePrincipals..")
titlePrincipals = pd.read_csv("/Users/nicolespaar/Desktop/title.principals.tsv", sep='\t', header=0, index_col='nconst')
#titlePrincipals.head(3)

titleCrew..
nameBasics..


In [8]:
def bugFixer(affectedList):
    '''
    bugFixer() receives a python list/array and checks each item to see if its 0th character
    is whitespace. If it is, the whitespace is removed and appended to a new list. If it is
    not, the item is appended to the same new list. The new list with no items' first
    character being whitespace is then returned.
    '''
    fixedList = []
    for item in affectedList:
        if item[0] == " ":
            fixedItem = item[1:]
            fixedList.append(fixedItem)
        else:
            fixedList.append(item)
    return fixedList

def lengths(x):
    if isinstance(x, list):
        yield len(x)
        for y in x:
            yield from lengths(y)

def iAmount(anArray):
    return max(lengths(anArray))

def init_splinter():
    '''
    init_splinter() initializes a selenium chrome webdriver using a Tor proxy and returns it
    as a splinter browser wrapped object.
    '''
    executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
    browser = Browser('chrome', **executable_path, headless=False, incognito=True)
    return browser

def simmer_soup():
    '''
    simmer_soup() receives the browser object and returns the current page's parsed html as "Soup".
    '''
    html = browser.html
    soup = bs(html, 'html.parser')
    return html, soup

#!---- ---- ---- ---- If $movieToSearch exists in db ---- ---- ---- ----!#
def tconstMunge():
    tconst = titleBasicsRow.tconst.to_string(index=False)[1:]
    return tconst

def genresMunge():
    genres_raw = titleBasicsRow.genres.to_string(index=False).split(",")
    genres = bugFixer(genres_raw)
    return genres

def entitiesMunge():
    titleCrewRow = titleCrew[titleCrew['tconst'] == tconst]
    crew_directors = titleCrewRow.directors.to_string(index=False).split(",")
    crew_writers = titleCrewRow.writers.to_string(index=False).split(",")
    titlePrincipalsRow = titlePrincipals[titlePrincipals['tconst'] == tconst]
    
    cast_actors = []
    cast_actors_jobs = []
    cast_other = []
    cast_other_jobs = []
    
    for row in titlePrincipalsRow.iterrows():
        x = row[1].category
        if (x in ('actor', 'actress')):
            cast_actors.append(row[0])
            
            cast_actors_jobs.append(row[1].job)
        else:
            cast_other.append(row[0])
            cast_other_jobs.append(row[1].job)
            
    return cast_actors, cast_actors_jobs, cast_other, cast_other_jobs, crew_directors, crew_writers
    
def entitiesMunge_Parser(cast_actors, cast_other, crew_directors, crew_writers):
    actor_names = []
    cast_names = []
    director_names = []
    writer_names = []
    
    for i in cast_actors:
        actor_names.append(nameBasics.loc[i].primaryName)
    for i in cast_other:
        cast_names.append(nameBasics.loc[i].primaryName)
    for i in crew_directors:
        director_names.append(nameBasics.loc[i].primaryName)
    for i in crew_writers:
        writer_names.append(nameBasics.loc[i].primaryName)
    
    return actor_names, cast_names, director_names, writer_names    
    
#!---- ---- ---- ---- etc ---- ---- ---- ----!#
bechdel_base = "http://bechdeltest.com/api/v1/getMoviesByTitle?title="
bechdel_bools = []

def callAPI(movieToSearch):
    conn = str(requests.get(bechdel_base + movieToSearch).json())
    if len(conn) <= 2:
        return 0
    else:
        return 1
    
def monthToNum(MonthName):
    return{
        'January': 1,
        'February': 2,
        'March': 3,
        'April': 4,
        'May': 5,
        'June': 6,
        'July': 7,
        'August': 8,
        'September': 9,
        'October': 10,
        'November': 11,
        'December': 12
    }[MonthName]

In [9]:
scraped = []

for row in winnersDF_Film.iterrows():
    movieToSearch = row[1].Entity
    movieYear = row[1].Year
    titleBasicsArray = titleBasics[titleBasics['lowercasedTitle'] == movieToSearch]
    titleBasicsRow_RAW = titleBasicsArray[titleBasicsArray['titleType'] == 'movie']
    titleBasicsRow = titleBasicsRow_RAW[titleBasicsRow_RAW['startYear'] == str(movieYear)]
    tconst = titleBasicsRow.index.to_list()
    tconst_str = ''.join(tconst)
    tconst_str = tconst_str[0:9]
    if len(tconst_str) != 0:
        temp = []
        temp.append(movieToSearch)
        temp.append(tconst_str) 
        temp.append(movieYear)
        time.sleep(.90)
        bechdelBool = callAPI(movieToSearch)
        temp.append(bechdelBool)
        scraped.append(temp)  
    else:
        temp = []
        time.sleep(1.78)
        base_url = "https://www.imdb.com/search/title/?title=" 
        movieToSearch = movieToSearch.replace(" ", "+")
        full_url = (base_url + movieToSearch)
        browser = init_splinter()
        browser.driver.minimize_window()
        browser.visit(full_url)
        html, soup = simmer_soup()
        movieToSearch = movieToSearch.replace("+", " ")
        results = soup.find_all('h3', class_='lister-item-header')
        for r in results:
            rYear = soup.select('h3 > span')[1].get_text(strip=True)
            rYear = rYear.replace("(", "")
            rYear = rYear.replace(")", "")
            if (r.a.text.lower() == movieToSearch) and movieToSearch not in temp:
                link_end = r.a['href']
                tconst_str = link_end[7:16]
                temp.append(movieToSearch)
                temp.append(tconst_str) 
                temp.append(movieYear)   
        browser.quit()
        time.sleep(.90)
        bechdelBool = callAPI(movieToSearch)
        temp.append(bechdelBool)
        scraped.append(temp)

scraped[0:10]

[['wings', 'tt0018578', 1927, 1],
 ['the broadway melody', 'tt0019729', 1928, 0],
 ['all quiet on the western front', 'tt0020629', 1929, 1],
 ['cimarron', 'tt0053715', 1930, 1],
 ['grand hotel', 'tt7671068', 1931, 1],
 ['cavalcade', 'tt0023876', 1932, 1],
 ['it happened one night', 'tt0025316', 1934, 1],
 ['mutiny on the bounty', 'tt0026752', 1935, 1],
 ['the great ziegfeld', 'tt0027698', 1936, 0],
 ['the life of emile zola', 'tt0029146', 1937, 0]]

In [10]:
scraped[46] = ['the godfather part ii', 'tt0071562', 1974, 0]

In [11]:
scraped[45:48]

[['the sting', 'tt0070735', 1973, 0],
 ['the godfather part ii', 'tt0071562', 1974, 0],
 ["one flew over the cuckoo's nest", 'tt0073486', 1975, 0]]

In [12]:
Model_finalTitles = []
Model_finalTconsts = []
Model_finalYear = []
Model_finalBechdel = []

for winner in scraped:
    Model_finalTitles.append(winner[0])
    Model_finalTconsts.append(winner[1])
    Model_finalYear.append(winner[2])
    Model_finalBechdel.append(winner[3])

model_data = {'TCONST': Model_finalTconsts, 'TITLE': Model_finalTitles, 'YEAR': Model_finalYear, 'BECHDEL': Model_finalBechdel}    
modelDF = pd.DataFrame(data=model_data)
modelDF.head(10)

,TCONST,TITLE,YEAR,BECHDEL
0,tt0018578,wings,1927,1
1,tt0019729,the broadway melody,1928,0
2,tt0020629,all quiet on the western front,1929,1
3,tt0053715,cimarron,1930,1
4,tt7671068,grand hotel,1931,1
5,tt0023876,cavalcade,1932,1
6,tt0025316,it happened one night,1934,1
7,tt0026752,mutiny on the bounty,1935,1
8,tt0027698,the great ziegfeld,1936,0
9,tt0029146,the life of emile zola,1937,0


In [13]:
scores = []
votecount = []
budgets = []
companies = []
releasedDays = []
releasedMonths = []
releasedYears = []
releasedCountries = []
goofs = []

imdb_base = "https://www.imdb.com/title/"
for pk in Model_finalTconsts:
    time.sleep(6.66)
    imdb_end = pk
    url = (imdb_base + imdb_end)
    browser = init_splinter()
    browser.visit(url)
    html, soup = simmer_soup()
    
    ratings_parent = soup.find('div', class_='ratingValue').strong['title']
    ratings = ratings_parent.split()
    score = ratings[0]
    
    #scores.append(pk)###
    
    scores.append(score)
    totalVotes = ratings[3]
    
    #votecount.append(pk)###
    
    votecount.append(totalVotes)
    
    cert_raw = soup.find_all('div', class_='txt-block')
    for cert in cert_raw:
        try:
            if cert.h4.text == "Budget:":
                budget_RAW = cert.text.split()
                budget = budget_RAW[0]
                budgets.append(pk)###
                budgets.append(budget[8:])
            elif cert.h4.text == "Production Co:":
                companies_RAW = cert.text.split()
                company = companies_RAW[2]            
                companies.append(company)
            elif cert.h4.text == "Release Date:":
                release_RAW = cert.text.split()
                release1 = release_RAW[2]
                release2 = release_RAW[3]
                release3 = release_RAW[4]
                release2 = monthToNum(release2)
             
                releasedDays.append(pk)###
                
                releasedDays.append(release1)
                
                releasedMonths.append(pk)###
                
                releasedMonths.append(release2)
                
                releasedYears.append(pk)###
                
                releasedYears.append(release3)
                release_country = release_RAW[5]
                release_country = release_country.replace("(", "")
                release_country = release_country.replace(")", "")
                
                releasedCountries.append(pk)###
                
                releasedCountries.append(release_country)
        except:
            pass 
    
    browser.quit()

In [14]:
budgets

['tt0018578',
 '2,000,000',
 'tt0019729',
 '379,000',
 'tt0020629',
 '1,448,864',
 'tt0053715',
 '5,421,000',
 'tt0023876',
 '1,180,280',
 'tt0025316',
 '325,000',
 'tt0026752',
 '1,950,000',
 'tt0030993',
 '1,644,736',
 'tt0031381',
 '3,977,000',
 'tt0032976',
 '1,288,000',
 'tt0033729',
 '1,250,000',
 'tt0035093',
 '1,344,000',
 'tt0034583',
 '950,000',
 'tt0037884',
 '1,250,000',
 'tt0036868',
 '2,100,000',
 'tt0039416',
 '2,000,000',
 'tt0040416',
 '500,000',
 'tt0042192',
 '1,400,000',
 'tt0043278',
 '2,723,903',
 'tt0044672',
 '4,000,000',
 'tt0045793',
 '1,650,000',
 'tt0047296',
 '910,000',
 'tt0048356',
 '343,000',
 'tt0048960',
 '6,000,000',
 'tt0050212',
 '3,000,000',
 'tt0051658',
 '3,319,355',
 'tt0052618',
 '15,900,000',
 'tt0053604',
 '3,000,000',
 'tt0055614',
 '6,000,000',
 'tt0056172',
 '15,000,000',
 'tt0057590',
 '1,000,000',
 'tt0058385',
 '17,000,000',
 'tt0059742',
 '8,200,000',
 'tt0060665',
 '2,000,000',
 'tt0061811',
 '2,000,000',
 'tt0063385',
 '10,000,000',


In [15]:
modelDF['RATING'] = scores
modelDF['VOTES'] = votecount
print(len(budgets))
modelDF['PRODUCER'] = companies
print(len(releasedDays))
print(len(releasedMonths))
print(len(releasedYears))
print(len(releasedCountries))

170
170
170
170
170


In [16]:
modelDF.head(10)

,TCONST,TITLE,YEAR,BECHDEL,RATING,VOTES,PRODUCER
0,tt0018578,wings,1927,1,7.7,"10,221",Paramount
1,tt0019729,the broadway melody,1928,0,6.2,"5,829",Metro-Goldwyn-Mayer
2,tt0020629,all quiet on the western front,1929,1,8.1,"53,696",Universal
3,tt0053715,cimarron,1930,1,6.4,"2,376",Metro-Goldwyn-Mayer
4,tt7671068,grand hotel,1931,1,6.6,"1,031",Unbelievable
5,tt0023876,cavalcade,1932,1,6.0,"4,014",Fox
6,tt0025316,it happened one night,1934,1,8.1,"85,416",Columbia
7,tt0026752,mutiny on the bounty,1935,1,7.8,"19,049",Metro-Goldwyn-Mayer
8,tt0027698,the great ziegfeld,1936,0,6.8,"6,438",Metro-Goldwyn-Mayer
9,tt0029146,the life of emile zola,1937,0,7.3,"6,322",Warner
